In [47]:
import os
from pprint import pprint

Запустить из консоли

opus_read --directory OpenSubtitles --source en --target ru -m 900000 --write result.txt --src_range 1-4 --tgt_range 1-4

https://github.com/Helsinki-NLP/OpusTools

In [48]:
with open("corpora/subtitles/ru-en_0-900.txt", "rt") as f:
    text = f.read()

data = text.split('================================')

In [49]:
data = [item.strip() for item in data]

ru = []
en = []
idx = 0
for item in data:
    if item.startswith('#') or not item:
        continue
    sentences = item.split('\n')
    src = []
    trg = []
    for sentence in sentences:
        # print(idx, sentence)
        if sentence.startswith("(src)"):
            trash, useful = sentence.split(">");
            useful = useful.split();
            src += useful
        else:
            trash, useful = sentence.split(">");
            useful = useful.split();
            trg += useful
        idx += 1
        
    ru.append(src)
    en.append(trg)


In [45]:
pprint(ru[:3])

[['Дети',
  'могут',
  'достать',
  'во',
  'дворе',
  'почти',
  'всё',
  'что',
  'угодно',
  'до',
  'тех',
  'пор',
  ',',
  'пока',
  'могут',
  'себе',
  'это',
  'позволить',
  '.'],
 ['Всё', 'имеет', 'свою', 'цену', '.'],
 ['Эй', ',', 'Ник', '.']]


In [46]:
pprint(en[:3])

[['Kids',
  'can',
  'get',
  'pretty',
  'much',
  'anything',
  'they',
  'want',
  'in',
  'the',
  'yard',
  ',',
  'as',
  'long',
  'as',
  'they',
  'can',
  'afford',
  'it',
  '.'],
 ["'Cause", 'everything', 'comes', 'with', 'a', 'price', '.'],
 ['Hey', ',', 'Nick', '.']]


In [40]:
import spacy
from tqdm import tqdm
from simalign import SentenceAligner

In [41]:
myaligner = SentenceAligner(model="bert", token_type="bpe", matching_methods="mai")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2023-01-13 22:17:08,273 - simalign.simalign - INFO - Initialized the EmbeddingLoader with model: bert-base-multilingual-cased


In [ ]:
chunk_size = 10000
latest_chunk = 0
for chunk_start in range(latest_chunk, len(ru), chunk_size):
    text = ""
    for i in tqdm(range(chunk_start, chunk_start + chunk_size)):
        src, trg = (ru[i], en[i])
        alignments = myaligner.get_word_aligns(src, trg)
        for first, second in alignments['itermax']:
            text += f"{src[first]}\t{trg[second]}\n"
        text += '\n'
    
    filename = f"corpora/subtitles/aligned_{chunk_start}-{chunk_start + chunk_size - 1}"
    with open(filename, "wt") as f:
        f.write(text)